In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
from supabase import create_client
#from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.output_parsers import StrOutputParser
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.schema import Document
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Literal
from typing import List
from typing_extensions import TypedDict
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from pprint import pprint
from typing import Any,  Literal, Union
from langchain_core.messages import  AnyMessage
from langchain.schema import AIMessage
from langchain_core.prompts import MessagesPlaceholder
import logging
import os
import load_data

Supabase client initialized: <supabase._sync.client.SyncClient object at 0x0000013C7FA70590>


In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_API_VERSION=os.getenv("OPENAI_API_VERSION")

vector_store=load_data.load_vector_store()
retriever=vector_store.as_retriever(search_kwargs={"k":4})
chat= AzureChatOpenAI(azure_deployment="gpt-4o-rfmanrique")


In [3]:
chat= AzureChatOpenAI(azure_deployment="gpt-4o-rfmanrique")



system="""
Eres un monitor asistente de la clase de introducción a la programación en Python. Tu función principal es responder dudas conceptuales que los estudiantes tengan sobre los diferentes módulos del curso. El curso está dividido en 4 módulos:

Módulo 1: Introducción a la programación

En este módulo, los estudiantes aprenden los fundamentos de la programación, cómo funcionan los lenguajes de programación y el proceso de escribir, ejecutar y depurar código.
Módulo 2: Condicionales

Aquí los estudiantes estudian las estructuras condicionales como if, else, y elif para tomar decisiones en el código.
Módulo 3: Bucles

En este módulo se cubren bucles for y while, junto con conceptos como control de bucles (break, continue).
Módulo 4: Librerías

Los estudiantes exploran cómo importar y utilizar librerías en Python, como math o random, y cómo instalar librerías externas.
Para responder preguntas de los estudiantes, tendrás como referencia el siguiente contexto tomado de un libro de programación:

{context}

Tu tarea es usar este contexto para responder de manera precisa, clara y útil a las preguntas. Intenta explicar los conceptos con ejemplos y lenguaje accesible para principiantes, proporcionando respuestas bien estructuradas y fáciles de entender. Si es necesario, usa fragmentos de código simples para ilustrar los conceptos.
El usuario se encuentra en el nivel : {level}
Utiliza el formato Markdown, incluyendo ‘ para código en línea.

Input del usuario : {user_input}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),      
        ("placeholder","{messages}")
    ]
)

conceptual_agent=prompt|chat

In [4]:
system = """
Eres un reformulador de preguntas que convierte una pregunta de entrada en una versión mejorada, optimizada para la búsqueda en una vector store. 
Observa la entrada y trata de razonar sobre la intención o significado semántico subyacente. La vector store trata tema sobre introducción a la programación.
En algunas ocasiones el estudiante hará follow-up questions, por lo que debes reformularla basado en los mensajes anteriores para que pueda entrar a la vector store.
Debes ser simple y conciso.

La pregunta es : {user_input}
"""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("placeholder","{messages}"),
    ]
)
chat= AzureChatOpenAI(azure_deployment="gpt-4o-rfmanrique")

question_rewriter = re_write_prompt | chat 

In [5]:

class State(TypedDict):
    messages:Annotated[list,add_messages]

In [7]:
def conceptual_assistant(state:State):
    user_input=state['messages'][-1]
    #Reformula pregunta para vector store
    query=question_rewriter.invoke({"user_input": user_input, "messages":state["messages"]}).content

    #Extrae contexto segun el query
    context=retriever.invoke(query)

    #Responde de acuerdo al contexto
    response=conceptual_agent.invoke({"user_input":user_input,"messages":state["messages"],"context":context,"level":"2"})
    return {"messages": [response]}

In [8]:
graph_builder=StateGraph(State)
#memory = SqliteSaver.from_conn_string(":memory:")
memory = MemorySaver()

In [9]:

graph_builder.add_node("conceptual_assistant",conceptual_assistant)
graph_builder.add_edge("conceptual_assistant",END)
graph_builder.add_edge(START,"conceptual_assistant")

In [12]:
graph = graph_builder.compile(checkpointer=memory)

config={"configurable":{"thread_id":1}}
lista=["hola", "Quiero saber mas de algebra booleana"]
for i in lista:
    for event in graph.stream({"messages": ("user", i),"level":2},config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)